<a href="https://colab.research.google.com/github/LaviJ/Cryptocurrency-Analysis/blob/main/Machine%20Learnnig/model-predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Forcasting Cryptocurrency
# Data Analysis and Visualization
# Group 7

# TensorFlow model prediction routine

import os
import datetime
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import pandas as pd
import seaborn as sns

symbols=['BTC','ETH','XRP','USDT','ADA','XMR','XLM','BUSD','LTC','DOGE']
modeltypes=['lstm','bidir','feedback']
testordinal='4'
for symbol in symbols:
  for modeltype in modeltypes:
    # Declare the dataframes
    train_df = val_df = test_df = df = pd.DataFrame()

    # Load the trained tensorflow model
    # TODO: make coin variable
    themodel = keras.models.load_model('hourlyCryptoModels/'+symbol+modeltype+'_model')

    # Use Keras to get the data file
    path = tf.keras.utils.get_file(
        origin='https://raw.githubusercontent.com/LaviJ/Cryptocurrency-Analysis/main/Data/Hourly/Test'+testordinal+'/test'+testordinal+'_'+symbol+'.csv',
        fname='test'+testordinal+'_'+symbol+'.csv')
    csv_path, _ = os.path.splitext(path)

    # Read data into dataframe
    df = pd.read_csv(csv_path+".csv")

    # Drop some columns
    df = df.drop('conversionType', axis=1)
    df = df.drop('conversionSymbol', axis=1)
    df = df.drop('volumeto', axis=1)
    df = df.drop('volumefrom', axis=1)
    df['mid'] = (df['high'] + df['low'])/2
    df = df.drop('open', axis=1)
    df = df.drop('high', axis=1)
    df = df.drop('low', axis=1)

    # Drop last row
    df = df[:-1]

    # Move the timestamps into an array
    date_time =[0]
    date_time = df.pop('time')
    firsttime = str(pd.to_datetime(date_time.iloc[-1],unit='s'))
    lasttime  = str(pd.to_datetime(date_time.iloc[0],unit='s'))

    column_indices = {name: i for i, name in enumerate(df.columns)}

    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    num_features = df.shape[1]

    MAX_EPOCHS = 11
    OUT_STEPS = 96

    # Show the input data
    df.plot(legend=True, figsize=(5,3), xlabel='Hours since '+firsttime+' UTC', ylabel='$ (USD)', colormap='coolwarm', title=symbol+': 96-hour Input Data', lw=3)
    plt.savefig('pngs/'+symbol+modeltype+'in.png',bbox_inches='tight',dpi=150)
    plt.close()
    # Adapt input array dimensions to fit trained model shape [None, 96, 8] 96 points of 8 dimensional data (2 price points, 6 periodic waveforms)
    
    # normalize dataset with MinMaxScaler
 
    scaler = MinMaxScaler(feature_range=(-2, 2))
    
    df['close']  = scaler.fit_transform(np.array(df['close']).reshape(-1, 1)) 
    df['mid']  = scaler.fit_transform(np.array(df['mid']).reshape(-1, 1)) 
    df['Daily Sine']=df['Daily Cosine']=df['Quarterly Sine']=df['Quaterly Cosine']= df['Annual Sine']=df['Annual Cosine']=np.zeros(96)
   
    answer=pd.DataFrame()
    # Run feedback model prediction on input data
    all = pd.DataFrame()
    first = np.zeros(96)
    for iter in range(96):
      x = tf.convert_to_tensor( df[:] )
      x = tf.expand_dims(x, 0, name="inputs")
      x = tf.ensure_shape(x, [1, 96, 8])
      with tf.device('/device:GPU:0'):
        answer = pd.DataFrame(themodel.predict(x,False)[0])

    # Put the answer into a dataframe
    final = pd.DataFrame( )
    final = answer
    # Show the raw componenet output
    final.columns = [ 'close','mid','Daily Sine','Daily Cosine','Quarterly Sine','Quarterly Cosine','Annual Sine','Annual Cosine']
    final.plot(legend=True, figsize=(5,3), xlabel='Hours since '+lasttime+' UTC', colormap='turbo', title=symbol+': Generated Components ('+modeltype+')', lw=4)
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.savefig('pngs/'+symbol+modeltype+'raw.png',bbox_inches='tight',dpi=150)
    plt.close()
    # Apply Weights and Denormalize

    final['close'] =  scaler.inverse_transform(np.array(final['close']*3).reshape(-1, 1))
    final['mid'] =  scaler.inverse_transform(np.array(final['mid']*2).reshape(-1, 1))
    final['Daily Sine'] =  scaler.inverse_transform(np.array(final['Daily Sine']*.05).reshape(-1, 1))
    final['Daily Cosine'] =  scaler.inverse_transform(np.array(final['Daily Cosine']*.05).reshape(-1, 1))
    final['Quarterly Sine'] =  scaler.inverse_transform(np.array(final['Quarterly Sine']*2).reshape(-1, 1))
    final['Quarterly Cosine'] =  scaler.inverse_transform(np.array(final['Quarterly Cosine']*2).reshape(-1, 1))
    final['Annual Sine'] =  scaler.inverse_transform(np.array(final['Annual Sine']*4).reshape(-1, 1))
    final['Annual Cosine'] =  scaler.inverse_transform(np.array(final['Annual Cosine']*4).reshape(-1, 1))
 
    # Show the weighted componenet output
    final.columns = [ 'close','mid','Daily Sine','Daily Cosine','Quarterly Sine','Quarterly Cosine','Annual Sine','Annual Cosine']
    final.plot(legend=True, figsize=(5,3), xlabel='Hours since '+lasttime+' UTC', colormap='Dark2', title=symbol+': Weighted Components ('+modeltype+')', lw=4)
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.savefig('pngs/'+symbol+modeltype+'w8s.png',bbox_inches='tight',dpi=150)
    plt.close()

    # Aggregate weighted predictions
    agg = np.ones(96)
    for i in range(96):
      agg[i] = np.mean(final.iloc[i])
    # Plot the aggregate prediction
    a=pd.DataFrame()
    a=pd.DataFrame( agg)

    ymin = float(np.min(a))
    xmin = int(np.where(a == ymin)[0][0])
    ymax = float(np.max(a))
    xmax = int(np.where(a == ymax)[0][0])
    if xmin < xmax :
        a.plot(legend=False, figsize=(5,3), xlabel='Hours since '+lasttime+' UTC\nmin @ \u03C4='+str(xmin)+', max @ \u03C4='+str(xmax)+'. \u0394= $'+str(round(ymax-ymin,7)), ylabel='$ (USD)', color='k', title=symbol+': Aggregate Predicted Trend ('+modeltype+')', lw=3)
    else:
        a.plot(legend=False, figsize=(5,3), xlabel='Hours since '+lasttime+' UTC\nmax @ \u03C4='+str(xmax)+', min @ \u03C4='+str(xmin)+'. \u0394= $\u2212'+str(abs(round(ymin-ymax,7))), ylabel='$ (USD)', color='k', title=symbol+': Aggregate Predicted Trend ('+modeltype+')', lw=3)

    plt.gca().scatter(xmin, ymin, s=123, marker='x')
    plt.gca().scatter(xmax, ymax, s=123, marker='x')
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%g'))
    plt.savefig('pngs/'+symbol+modeltype+'out.png', bbox_inches='tight', dpi=150)
    plt.close()

    print(symbol+' '+modeltype+' '+testordinal)

BTC lstm 4
BTC bidir 4
BTC feedback 4
ETH lstm 4
ETH bidir 4
ETH feedback 4
XRP lstm 4
XRP bidir 4
XRP feedback 4
USDT lstm 4
USDT bidir 4
USDT feedback 4
ADA lstm 4
ADA bidir 4
ADA feedback 4
XMR lstm 4
XMR bidir 4
XMR feedback 4
XLM lstm 4
XLM bidir 4
XLM feedback 4
BUSD lstm 4
BUSD bidir 4
BUSD feedback 4
LTC lstm 4
LTC bidir 4
LTC feedback 4
DOGE lstm 4
DOGE bidir 4
DOGE feedback 4


    # Tensorflow Routines for Time Series Data
    # https://www.tensorflow.org/tutorials/structured_data/time_series

    '''Martín Abadi, Ashish Agarwal, Paul Barham, Eugene Brevdo,
    Zhifeng Chen, Craig Citro, Greg S. Corrado, Andy Davis,
    Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Ian Goodfellow,
    Andrew Harp, Geoffrey Irving, Michael Isard, Rafal Jozefowicz, Yangqing Jia,
    Lukasz Kaiser, Manjunath Kudlur, Josh Levenberg, Dan Mané, Mike Schuster,
    Rajat Monga, Sherry Moore, Derek Murray, Chris Olah, Jonathon Shlens,
    Benoit Steiner, Ilya Sutskever, Kunal Talwar, Paul Tucker,
    Vincent Vanhoucke, Vijay Vasudevan, Fernanda Viégas,
    Oriol Vinyals, Pete Warden, Martin Wattenberg, Martin Wicke,
    Yuan Yu, and Xiaoqiang Zheng.'''

    # TensorFlow: Large-scale machine learning on heterogeneous systems,
    # 2015. Software available from tensorflow.org.